<a href="https://colab.research.google.com/github/DamodaraBarbosa/infos_xbox_game_pass/blob/main/Web_Scraping_Xbox_Series_Metacritic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importando as bibliotecas que serão utilizadas


Pandas, urllib e bs4

In [ ]:
import pandas as pd
from urllib.request import urlopen, Request
from urllib.error import HTTPError, URLError
from bs4 import BeautifulSoup

## Definindo algumas funções

Função para obter o metascore do jogo

In [ ]:
# função para obtenção da nota do metascore

def metascore_score(mscore):
  if 'positive' in str(mscore):
    meta = mscore.find('div', {'class': 'metascore_w large game positive'}).get_text()
  elif 'mixed' in str(mscore):
    meta = mscore.find('div', {'class': 'metascore_w large game mixed'}).get_text()
  else:
    meta = mscore.find('div', {'class': 'metascore_w large game negative'}).get_text()
  
  return meta

Função para obter o userscore do jogo

In [ ]:
# função para obtenção da nota do userscore

def userscore_score(uscore): 
  if 'positive' in str(uscore):
    user = uscore.find('div', {'class': 'metascore_w user large game positive'}).get_text()
  elif 'mixed' in str(uscore):
    user = uscore.find('div', {'class': 'metascore_w user large game mixed'}).get_text()
  elif 'negative' in str(uscore):
    user = uscore.find('div', {'class': 'metascore_w user large game negative'}).get_text()
  else:
    user = uscore.find('div', {'class': 'metascore_w user large game tbd'}).get_text()
  
  return user

## Obtenção dos dados

Criando rotina de scraping

In [ ]:
# o dado de cada jogo será armazenado no dicionário 'game' e cada 'game' na lista 'games'

game = dict()
games = list()

In [ ]:
for i in range(0, 3):
  headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'}

  try:
    req = Request(('https://www.metacritic.com/browse/games/release-date/available/xbox-series-x/metascore?page=' + str(i)), headers = headers)
    response = urlopen(req)
    html = response.read().decode('utf-8')
    soup = BeautifulSoup(html, 'html.parser')
  except HTTPError as e:
    print(e.status, e.reason)
  except URLError as e:
    print(e.reason)

  soup_jogos = soup.findAll('td', {'class': 'clamp-summary-wrap'})

  for index, soup_jogo in enumerate(soup_jogos):
    game['Name'] = soup_jogo.find('a', {'class': 'title'}).find('h3').get_text()
    spans = soup_jogo.find('div', {'class': 'clamp-details'}).findAll('span')
    game['Release date'] = spans[-1].get_text()

    metascore = soup_jogo.find('div', {'class': 'clamp-metascore'})
    userscore = soup_jogo.find('div', {'class': 'clamp-userscore'})
    game['Metascore'] = metascore_score(metascore)
    game['Userscore'] = userscore_score(userscore)

    games.append(game.copy())

In [ ]:
game

{'Name': 'Foreclosed',
 'Release date': 'August 12, 2021',
 'Metascore': '59',
 'Userscore': '5.8'}

In [ ]:
len(games)

300

## Exportanto os dados como Dataframe

Gerando o Dataframe

In [ ]:
df_xbox_series_metacritic = pd.DataFrame(games)

In [ ]:
df_xbox_series_metacritic

,Name,Release date,Metascore,Userscore
0,Elden Ring,"February 25, 2022",96,7.7
1,Persona 5 Royal,"October 21, 2022",95,8.5
2,Hades,"August 13, 2021",93,8.6
3,The Stanley Parable: Ultra Deluxe,"April 27, 2022",93,8.5
4,Forza Horizon 5,"November 5, 2021",92,8.2
...,...,...,...,...
295,Fatal Frame: Maiden of Black Water,"October 28, 2021",60,9.3
296,Morkredd,"December 10, 2020",60,5.7
297,Rugby 22,"January 27, 2022",59,tbd
298,Matchpoint: Tennis Championships,"July 7, 2022",59,5.9


In [ ]:
df_xbox_series_metacritic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Name          300 non-null    object
 1   Release date  300 non-null    object
 2   Metascore     300 non-null    object
 3   Userscore     300 non-null    object
dtypes: object(4)
memory usage: 9.5+ KB


Exportando o Dataframe

In [ ]:
df_xbox_series_metacritic.to_csv('df_xbox_series_metacritic.csv', index = False, encoding = 'utf-8-sig')